In [ ]:
print(data["lidar"][0:8])

In [ ]:
import matplotlib.pyplot as plt
data["lidar"].hist(bins = max(data["lidar"]) - min(data["lidar"]),align='left')
plt.show()

In [ ]:
data["lidar"].hist(bins = max(data["lidar"]) - min(data["lidar"]),color="orange",align='left')
plt.vlines(mean1,ymin=0,ymax=5000,color="red")
plt.show()

In [ ]:
# 定義から計算
zs = data["lidar"].values
mean = sum(zs)/len(zs)
diff_square = [(z-mean)**2 for z in zs]

sampling_var = sum(diff_square)/(len(zs))   # 標本分散
unbiased_var = sum(diff_square)/(len(zs)-1) # 不偏分散

print(sampling_var)
print(unbiased_var)

# Pandasを使用
pandas_sampling_var = data["lidar"].var(ddof = 0)   # 標本分散
pandas_default_var = data["lidar"].var()            # デフォルト（不偏分散）

print(pandas_sampling_var)
print(pandas_default_var)

# NumPyを使用
import numpy as np

numpy_default_var = np.var(data["lidar"])               # デフォルト（標本分散）
numpy_unbiased_var = np.var(data["lidar"], ddof = 1)    # 不偏分散

print(numpy_default_var)
print(numpy_unbiased_var)


In [ ]:
import math

# 定義から計算
stddev1 = math.sqrt(sampling_var)
stdde
